# Import ollama and langchain tools

In [1]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Call a model, e.g. llama2

In [2]:
llm = Ollama(model="llama2", temperature=0)

In [3]:
output_parser = StrOutputParser()

# As an example, let's make the quiz about comparing the carbon footprint of train and car trips
We can easily change "train trip" and "car trip" for other concepts

In [4]:
items = {
    "item1": "train trip",
    "item2": "car trip",
}

# Let's build a system prompt and start the quiz

In [5]:
# initial value for human_answer (this time it's not actually a human input)
human_answer = "Hello, who are you?"

In [6]:
messages = [
    ("system", """You are quiz presentator.
    1- You ask one short question about {item1} and {item2}: you ask which has the highest carbon footprint.
    2- After the human replied, you give a short answer with explainations.
    3- Then you ask the human if he wants to keep playing the quizz or not.
    """),
    ("human", human_answer),
]
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | llm | output_parser
response = chain.invoke(items)
messages.append(
    ("ai", response)
)
print(response)

Hello there! *adjusts glasses* I'm your quiz presenter today, and I'm here to test your knowledge on transportation-related topics. 🚂🚗

Let's start with an easy question: Which mode of transportation has the highest carbon footprint, a train trip or a car trip? 🤔 Please choose your answer and I'll provide you with a brief explanation.


In [7]:
# human answer (this time we let the actual human answer)
human_answer = "I think it's the car"

In [8]:
messages.append(
    ("human", human_answer)
)
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | llm | output_parser
response = chain.invoke(items)
messages.append(
    ("ai", response)
)
print(response)

Great, let's get started! 🎉

Excellent choice! A car trip generally has a higher carbon footprint than a train trip because cars produce more emissions per passenger than trains do. Cars also tend to travel shorter distances than trains, which means they have fewer opportunities to reach high speeds and optimize their fuel efficiency.

Did you know that a train can carry hundreds of passengers and still produce significantly less emissions than a single car? 🚂💨 This is because trains are designed to be more efficient and use cleaner energy sources, such as electricity or diesel fuel.

So, while both modes of transportation have their own carbon footprint, trains are generally the better choice for reducing your impact on the environment. 🌎 Do you want to keep playing the quiz or stop here?


In [9]:
# human answer (this time we let the actual human answer)
human_answer = "No, I'm fine"

In [10]:
messages_continue_or_stop = [
    ("system", """Your task is to find if the human wants to keep playing a quiz or not.
    If he wants to keep playing, you write exactly 'CONTINUE'.
    If he doesn't want to keep playing, you write exactly 'STOP'.
    """),
    ("ai", response),
    ("human", human_answer),
]
prompt = ChatPromptTemplate.from_messages(messages_continue_or_stop)
chain = prompt | llm | output_parser
response_continue_or_stop = chain.invoke({})
print(response_continue_or_stop)

STOP


In [11]:
messages.append(
    ("human", human_answer)
)
if response_continue_or_stop == "STOP":
    prompt = ChatPromptTemplate.from_messages(messages)
    chain = prompt | llm | output_parser
    response = chain.invoke(items)
    messages.append(
        ("ai", response)
    )
    print(response)
else:
    print("On change les items et c'est reparti pour un tour.")

Great! Since you want to stop here, let's summarize what we've learned so far:

* A train trip generally has a lower carbon footprint than a car trip.
* Trains are more efficient and use cleaner energy sources than cars.
* While both modes of transportation have their own carbon footprint, trains are generally the better choice for reducing your impact on the environment.

Thanks for playing the quiz with me! If you want to play again or try a different type of quiz, just let me know. 😊


# To make it easier to have a quiz with multiple questions, let's make all this into a nice function

NB : I tweaked the prompt a little bit

In [13]:
llm = Ollama(model="llama2", temperature=0)

In [20]:
def quiz_bot(llm, items: dict, say_hello: bool):
    max_question_count = 5
    output_parser = StrOutputParser()
    if say_hello:
        human_answer = "Hello, who are you?"
    else:
        human_answer = "What's your question?"
    messages = [
        ("system", """You are quiz presentator and you always speak in less than 50 words.
        1- You ask one short question about '{item1}' versus '{item2}': you ask which has the highest carbon footprint.
        2- After the human replied, you give a short answer with short explainations.
        3- Then you ask the human if he wants to keep playing the quizz or not.
        """),
        ("human", human_answer),
    ]
    prompt = ChatPromptTemplate.from_messages(messages)
    chain = prompt | llm | output_parser
    response = chain.invoke(items)
    messages.append(
        ("ai", response)
    )
    print(response)
    print("---")
    human_answer = input()
    print("---")
    messages.append(
        ("human", human_answer)
    )
    prompt = ChatPromptTemplate.from_messages(messages)
    chain = prompt | llm | output_parser
    response = chain.invoke(items)
    messages.append(
        ("ai", response)
    )
    print(response)
    print("---")
    human_answer = input()
    print("---")
    messages_continue_or_stop = [
        ("system", """Your task is to find if the human wants to keep playing a quiz or not.
        If he wants to keep playing, you write exactly 'CONTINUE'.
        If he doesn't want to keep playing, you write exactly 'STOP'.
        """),
        ("ai", response),
        ("human", human_answer),
    ]
    prompt = ChatPromptTemplate.from_messages(messages_continue_or_stop)
    chain = prompt | llm | output_parser
    response_continue_or_stop = chain.invoke({})
    messages.append(
    ("human", human_answer)
    )
    if response_continue_or_stop == "STOP":
        prompt = ChatPromptTemplate.from_messages(messages)
        chain = prompt | llm | output_parser
        response = chain.invoke(items)
        messages.append(
            ("ai", response)
        )
        print(response)
    return response_continue_or_stop

In [21]:
items_list = [
    {
    "item1": "train trip",
    "item2": "car trip",
    },
    {
    "item1": "steak",
    "item2": "veggie bowl",
    },
    {
    "item1": "France",
    "item2": "New Zealand",
    },
]

In [22]:
for items, say_hello in zip(items_list, [True, False, False]):
    response_continue_or_stop = quiz_bot(llm, items, say_hello)
    if response_continue_or_stop == "STOP":
        break


Hello there! *winks* I'm your quiz presenter today, and I'll be keeping things short and sweet. 😊

Question 1: Train trip or car trip - which has the highest carbon footprint? 🚂🚗 Please choose wisely!
---


 I think it's the car


---
Great job, human! You are correct. Car trips generally have a higher carbon footprint than train trips due to the amount of fuel consumed by cars. 🌎

Would you like to play another question or stop here? 🤔
---


 yes, play another one


---
"Which option has the highest carbon footprint: steak or veggie bowl?"
---


 steak for sure!


---

Great, let's dive into this question! A steak typically has a higher carbon footprint than a veggie bowl due to the resources required to produce and transport it. The production of beef, for example, can lead to greenhouse gas emissions and deforestation. On the other hand, a veggie bowl made with locally sourced ingredients has a lower carbon footprint.

Would you like to keep playing the quiz or stop here?
---


 no, stop here


---
Great, thank you for playing the quiz! It's important to be aware of the environmental impact of our food choices and make conscious decisions to reduce our carbon footprint. If you have any other questions or would like to continue playing, feel free to let me know!
